In [39]:

import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, date_trunc, count
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

# ------------------------------------------------------------
# Step 1: Load Excel data into a dictionary of DataFrames
# ------------------------------------------------------------
file = "/Users/anmolray/Library/CloudStorage/OneDrive-UniversityofWaterloo/InterviewPrep/Coffee Shop Sales:Inventory:Staff/CoffeeShop.xlsx"
xl = pd.ExcelFile(file)
d = {}
for sheet in xl.sheet_names:
    d[sheet] = pd.read_excel(xl, sheet_name=sheet)



In [ ]:

# Extract the orders data from the dictionary

if "orders" not in d:
    raise ValueError("The Excel file does not contain a sheet named 'orders'.")
orders_pd = d["orders"]
orders_pd = orders_pd.rename(columns={'created_at': 'order_time'}) 
# Verify that the orders DataFrame has the exact column needed: "order_time"
if "order_time" not in orders_pd.columns:
    raise ValueError("The 'orders' sheet must contain an 'order_time' column.")


In [ ]:
# Spark DataFrame and aggregate orders by hour

spark = SparkSession.builder.appName("OccupancyForecasting").getOrCreate()

# Create Spark DataFrame from the orders pandas DataFrame
orders_df = spark.createDataFrame(orders_pd)

# Convert the order_time column to a timestamp (using the exact column name)
orders_df = orders_df.withColumn("order_time", to_timestamp("order_time"))

# Aggregate by hour: count orders per hour as a proxy for occupancy
hourly_orders = (orders_df
                 .groupBy(date_trunc("hour", orders_df.order_time).alias("hour"))
                 .agg(count("*").alias("orders_count"))
                 .orderBy("hour"))

# Convert the aggregated Spark DataFrame to a pandas DataFrame for further processing
data = hourly_orders.toPandas()
data['hour'] = pd.to_datetime(data['hour'])
data = data.sort_values("hour")
print("Aggregated occupancy data:")
print(data)

Aggregated occupancy data:
                  hour  orders_count
0  2024-02-12 07:00:00            17
1  2024-02-12 08:00:00            16
2  2024-02-12 09:00:00            10
3  2024-02-12 10:00:00             4
4  2024-02-12 11:00:00             3
..                 ...           ...
61 2024-02-17 13:00:00            11
62 2024-02-17 14:00:00             9
63 2024-02-17 15:00:00             5
64 2024-02-17 16:00:00             4
65 2024-02-17 17:00:00             1

[66 rows x 2 columns]


In [ ]:
# Data Preparation for Time Series
# Use the exact "orders_count" column produced by our aggregation as our time series
occupancy_series = data['orders_count'].values.astype(float)

# Normalize the time series
scaler = MinMaxScaler(feature_range=(0, 1))
occupancy_scaled = scaler.fit_transform(occupancy_series.reshape(-1, 1))

def create_dataset(dataset, seq_length):
    """
    Creates input/output pairs for time series forecasting.
    Each input sequence consists of 'seq_length' consecutive time steps
    and the corresponding target is the immediate next value.
    """
    X, y = [], []
    for i in range(len(dataset) - seq_length):
        X.append(dataset[i:i+seq_length])
        y.append(dataset[i+seq_length])
    return np.array(X), np.array(y)

# Use the past 2 hours to predict the next hour
seq_length = 2  
X, y = create_dataset(occupancy_scaled, seq_length)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)  # Shape: (num_samples, seq_length)
y_tensor = torch.tensor(y, dtype=torch.float32)  # Shape: (num_samples, 1)



In [ ]:
#  PyTorch LSTM forecasting model

class LSTMForecast(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1):
        super(LSTMForecast, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # Initialize hidden and cell states with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Forward propagate LSTM; x shape should be (batch_size, seq_length, input_size)
        out, _ = self.lstm(x, (h0, c0))
        # Use the output of the last time step for forecasting
        out = self.fc(out[:, -1, :])
        return out

model = LSTMForecast()


In [48]:
#  Train LSTM model

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 1000  # Adjust the number of epochs as needed
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    # Ensure input tensor has shape: (batch_size, seq_length, input_size)
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")



Epoch [10/1000], Loss: 0.0447
Epoch [20/1000], Loss: 0.0447
Epoch [30/1000], Loss: 0.0447
Epoch [40/1000], Loss: 0.0447
Epoch [50/1000], Loss: 0.0447
Epoch [60/1000], Loss: 0.0447
Epoch [70/1000], Loss: 0.0447
Epoch [80/1000], Loss: 0.0447
Epoch [90/1000], Loss: 0.0447
Epoch [100/1000], Loss: 0.0447
Epoch [110/1000], Loss: 0.0447
Epoch [120/1000], Loss: 0.0447
Epoch [130/1000], Loss: 0.0447
Epoch [140/1000], Loss: 0.0447
Epoch [150/1000], Loss: 0.0447
Epoch [160/1000], Loss: 0.0447
Epoch [170/1000], Loss: 0.0447
Epoch [180/1000], Loss: 0.0447
Epoch [190/1000], Loss: 0.0447
Epoch [200/1000], Loss: 0.0447
Epoch [210/1000], Loss: 0.0447
Epoch [220/1000], Loss: 0.0447
Epoch [230/1000], Loss: 0.0447
Epoch [240/1000], Loss: 0.0447
Epoch [250/1000], Loss: 0.0447
Epoch [260/1000], Loss: 0.0447
Epoch [270/1000], Loss: 0.0447
Epoch [280/1000], Loss: 0.0447
Epoch [290/1000], Loss: 0.0447
Epoch [300/1000], Loss: 0.0447
Epoch [310/1000], Loss: 0.0447
Epoch [320/1000], Loss: 0.0447
Epoch [330/1000],

In [ ]:
#  Forecast the next hour's customer occupancy

model.eval()
# Prepare the last sequence from the scaled data for prediction
last_seq = torch.tensor(occupancy_scaled[-seq_length:], dtype=torch.float32).unsqueeze(0)
predicted = model(last_seq).detach().numpy()
# Convert the prediction back to the original scale
predicted_occupancy = scaler.inverse_transform(predicted)
print("\nPredicted occupancy for next hour:", predicted_occupancy[0][0])




Predicted occupancy for next hour: 7.626443


In [ ]:
#optimize staffing based on the prediction

# Using a simple rule: one staff per 5 customers, with a minimum of 2 staff members.
predicted_customers = predicted_occupancy[0][0]
staff_needed = max(2, int(np.ceil(predicted_customers / 5)))
print("Optimized staff count for next hour:", staff_needed)

# Stop the Spark session when done
spark.stop()

Optimized staff count for next hour: 2
